In [2]:
import cv2
import os
import time
from memory_queues import square_queue, gui_queue
from memory_logic import register_card
from sift_utils import extract_sift_signature, auto_crop_inside_white_edges, draw_oriented_bounding_box
from recorded_positions import pick_positions, drop_positions, home_pose, scan_pose
from pyniryo2 import NiryoRos, NiryoRobot, Vision
import pyniryo
from matplotlib import pyplot as plt
import numpy as np

Connected to server (169.254.200.200) on port 40001
/var/folders/02/s2j6q52d0k5c5x3kf7hlyzcr0000gn/T/ipykernel_7990/1266433035.py:8: DeprecationWarning: This project is officially marked as deprecated and will no longer be maintained or receive updates starting 1st June 2025. We recommend users migrate to PyNiryo: https://pypi.org/project/pyniryo/
  from pyniryo2 import NiryoRos, NiryoRobot, Vision


In [3]:
robot_ip_address = "169.254.200.200"
robot = NiryoRobot(robot_ip_address)
robot.arm.calibrate_auto()
# robot.activate_vacuum()
robot.tool.release_with_tool()
robot.arm.move_pose(home_pose)


In [9]:
pose_read = robot.arm.get_pose()
print(f'{round(pose_read.x,2), round(pose_read.y,2), round(pose_read.z,2), round(pose_read.roll,2), round(pose_read.pitch,2), round(pose_read.yaw,4)} ')

(0.24, -0.22, 0.04, 2.59, 1.54, 1.8409) 


In [ ]:
#L1
robot.arm.move_pose([0.32, 0.21, 0.038, 2.82, 1.48, 3.1381])

In [22]:
#home pose 
robot.arm.move_pose([0.33, 0.01, 0.19, 0.25, 1.45, 0.314])

In [5]:
#scan pose
robot.arm.move_pose([0.44, 0.04, 0.25, 1.59, 0.05, 1.2969586785279166])

In [ ]:
robot.arm.move_pose([0.38, 0.15, 0.05, 0.87, 1.51, 0.9])

In [20]:
robot.tool.grasp_with_tool()

In [23]:
robot.tool.release_with_tool()


In [ ]:
robot.tool.release_with_tool()
robot.tool.grasp_with_tool()
robot.arm.move_pose([-0.02, -0.46, 0.18, -0.13, -1.15, -1.504])


In [ ]:

import cv2
CARD_BOX = (270,140,190,190)  #x,y,h,w

ros_instance = NiryoRos("169.254.200.200")
vision = Vision(ros_instance)

print(f"[SCAN] Looking for card at ")
last_center = stable_since = detection_time = None
while True:
    img_compressed = vision.get_img_compressed()
    img_uncompressed = pyniryo.uncompress_image(img_compressed)
    camera_info = vision.get_camera_intrinsics()
    img = pyniryo.undistort_image(img_uncompressed, camera_info.intrinsics, camera_info.distortion)

    frame = cv2.resize(img, (640, 480))
    cv2.rectangle(frame, (CARD_BOX[0], CARD_BOX[1]), (CARD_BOX[0]+CARD_BOX[2], CARD_BOX[1]+CARD_BOX[3]), (0, 255, 0), 2)
    cv2.imshow("Adjust ROI", frame)

    key = cv2.waitKey(1)
    if key == ord('q'): break


cv2.destroyAllWindows()

[SCAN] Looking for card at 


KeyboardInterrupt: 

: 